This notebook will read a retrieve a list of infant SNe as described in a series of input lists, and uses these to create a permabent alert topic. This can later be used to repeatedly retrieve the alerts.

Targets will be restricted to _names_ belonging to 2020 to create a controlled sample (with dRB, references etc). This will allow the creation of filter that works in the future, but might not work from the start of ZTF.

In [ ]:
import os, requests
import pandas as pd
from ampel_notebook_utils import api_name2candids
import matplotlib.pyplot as plt

In [ ]:
# Early SNe Ia, as collected by M.D.
df_ia = pd.read_csv('/home/jnordin/data/ztf/filter_infant_sne/earlyia_maxime.dat', header=None, names=['name'])
sne = list(df_ia.name)

In [ ]:
# Early CCSN, as collected by RB. 
# We here use the early 
df_cc = pd.read_csv('/home/jnordin/data/ztf/filter_infant_sne/fup_within_1d_sneII.ascii')
sne.extend( list(df_cc['name']) )

In [ ]:
df_cc = pd.read_csv('/home/jnordin/data/ztf/filter_infant_sne/fup_m1d_l2d_sneII.ascii')
sne.extend( list(df_cc['name']) )

In [ ]:
# Distribution per year prior to limit
_ = plt.hist([int(sn[3:5]) for sn in sne])
plt.xlabel('Year of ZTF name')

In [ ]:
sne = [sn for sn in sne if sn[3:5]=='20']

In [ ]:
print('We will create a topic with {} supernovae'.format(len(sne)))

A AMPEL archive access token needs to be supplied to access the archive. This can be obtained from the archive tab of 
https://ampel.zeuthen.desy.de/live/dashboard/tokens
(assumes user to have joined the AmpelProject github organization). 

In [ ]:
token = os.environ["ARCHIVE_TOKEN"]   # Token here stored as system variable

In [ ]:
# Find alert candidate IDS for the list of sne
# Takes a while since this queries the archive for each
candids = [api_name2candids(sn, token) for sn in sne]

In [ ]:
mycandids = [a for sncandids in candids for a in sncandids]

In [ ]:
print('Preparing to create a topic containing {} alerts from {} sne'.format(len(mycandids), len(candids)))

Post topics based on this list. Start by one containing the most populous alert from each.

In [ ]:
topic_disc = 'A list of 2565 alerts from 27 2020 SNe which were detected early. Created May 24 2022.'

In [ ]:
body = {
    "description" : topic_disc,
    "candids" : mycandids
}

In [ ]:
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'

In [ ]:
response = requests.post(f"{base_url}/topics/", headers={"Authorization": "bearer "+token}, json=body )

In [ ]:
topic_token = response.json()

In [ ]:
print(topic_token)

This is now a _permanent_ token which should be appropriately stored. Note that it is not possible retrieve a token once created through the api, hence you need to keep track of it.

*** 
_topic_:  Y9R82mkcL1SJq16rTt8_vh-_dx0V7rwEHbGgSgUMlqI

_description_: A list of the final alert of 27 SNe which were detected early during 2020. Created May 24 2022.
***

_topic_:  fz6nmCJwKSdZXhzm3JuBTlMY3xjPsdhkobVLvZzzmt8

_description_: A list of 2565 alerts from 27 2020 SNe which were detected early. Created May 24 2022.

